# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'hear_failure_prediction'

experiment=Experiment(ws, experiment_name)
experiment

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "iterations": 40,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}


# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_metrics = best_run.get_metrics()

print("Best Run Id : {}".format(best_run.id), 
      "Accuracy : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n')

In [ ]:
print(fitted_model._final_estimator)

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name = "heart-failure-predict-best-model-automl", model_path = './outputs/model.pkl')
print(model)

In [ ]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Save the best model for the deployement
import os
os.makedirs('./aml-model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./aml-model','heart-failure-predict-best-model-automl.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./aml-model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [ ]:
model=best_run.register_model(
            model_name = 'heart-failure-best-model-automl', 
            model_path = './outputs/model.pkl'
)

## Set up the environment

In [ ]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-env', file_path = 'conda_env.yml')

## Download the scoring file

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

## Inference Config

In [ ]:
inference_config = InferenceConfig(entry_script= 'score.py', environment=environment)

## ACI config

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

## Deploy the model

In [ ]:
webservice = Model.deploy(workspace=ws, 
                        name='heart-failure-ws', 
                        models=[model], 
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)

In [ ]:
webservice

In [ ]:
# wait for deployment to finish and display the scoring uri and swagger uri
webservice.wait_for_deployment(show_output=True)

print("Status : {}".format(webservice.state), 
      "scoring_uri : {}".format(webservice.scoring_uri), 
      "swagger_uri : {}".format(webservice.swagger_uri), sep = '\n')

## Consume

In [ ]:
from pprint import pprint

# select samples from the dataframe
test_data = df.sample(5)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})
pprint(data)

## Get ineference

In [ ]:
output = webservice.run(data)
print(output)

In [ ]:

webservice.get_logs()

In [ ]:
# select samples from the dataframe
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})

# get inference
output = webservice.run(data)
print(output)

In [ ]:
webservice.get_logs()

In [ ]:
webservice.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
